In [1]:
!apt-get update
!apt-get install clang libomp-dev

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [808 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,082 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ub

In [6]:
%%writefile hello2.cpp

#include <stdio.h>
#include <stdlib.h>
#include <time.h>

#define INPUT_SIZE 1000
#define HIDDEN_SIZE 256
#define OUTPUT_SIZE 100
#define LEARNING_RATE 0.001
#define ITERATIONS 500

// Define neural network parameters
float input[INPUT_SIZE];
float hidden_weights[INPUT_SIZE][HIDDEN_SIZE];
float hidden_bias[HIDDEN_SIZE];
float hidden_output[HIDDEN_SIZE];
float output_weights[HIDDEN_SIZE][OUTPUT_SIZE];
float output_bias[OUTPUT_SIZE];
float output[OUTPUT_SIZE];

// Forward pass of the neural network
void forward_pass() {
    // Compute hidden layer output in parallel
    #pragma omp parallel for
    for (int i = 0; i < HIDDEN_SIZE; i++) {
        float sum = 0.0;
        for (int j = 0; j < INPUT_SIZE; j++) {
            sum += input[j] * hidden_weights[j][i];
        }
        hidden_output[i] = sum + hidden_bias[i];
        hidden_output[i] = (hidden_output[i] > 0) ? hidden_output[i] : 0; // ReLU activation
    }

    // Compute output layer output in parallel
    #pragma omp parallel for
    for (int i = 0; i < OUTPUT_SIZE; i++) {
        float sum = 0.0;
        for (int j = 0; j < HIDDEN_SIZE; j++) {
            sum += hidden_output[j] * output_weights[j][i];
        }
        output[i] = sum + output_bias[i];
    }
}

// Backward pass of the neural network
void backward_pass(float target[OUTPUT_SIZE]) {
    // Compute output layer gradients in parallel
    #pragma omp parallel for
    for (int i = 0; i < OUTPUT_SIZE; i++) {
        float error = output[i] - target[i];
        for (int j = 0; j < HIDDEN_SIZE; j++) {
            output_weights[j][i] -= LEARNING_RATE * error * hidden_output[j];
        }
        output_bias[i] -= LEARNING_RATE * error;
    }

    // Compute hidden layer gradients in parallel
    #pragma omp parallel for
    for (int i = 0; i < HIDDEN_SIZE; i++) {
        float error = 0.0;
        for (int j = 0; j < OUTPUT_SIZE; j++) {
            error += (output[j] - target[j]) * output_weights[i][j];
        }
        error *= (hidden_output[i] > 0) ? 1 : 0; // ReLU derivative
        for (int j = 0; j < INPUT_SIZE; j++) {
            hidden_weights[j][i] -= LEARNING_RATE * error * input[j];
        }
        hidden_bias[i] -= LEARNING_RATE * error;
    }
}

int main() {
    // Initialize neural network parameters
    // (Initialization code omitted for brevity)

    // Dummy input (for demonstration purposes)
    for (int i = 0; i < INPUT_SIZE; i++) {
        input[i] = (float)(rand()) / RAND_MAX;
    }

    // Dummy target output (for demonstration purposes)
    float target[OUTPUT_SIZE] = {0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0};

    // Timing code
    clock_t start_time = clock();

    // Perform multiple iterations of forward and backward passes
    for (int iter = 0; iter < ITERATIONS; iter++) {
        forward_pass();
        backward_pass(target);
    }

    clock_t end_time = clock();
    double total_time = (double)(end_time - start_time) / CLOCKS_PER_SEC;
    printf("Total time: %f seconds\n", total_time);

    return 0;
}


Overwriting hello2.cpp


In [10]:
!g++ hello2.cpp -o hello2
!./hello2

Total time: 3.231915 seconds


In [12]:
import matplotlib.pyplot as plt
from tabulate import tabulate

# Number of threads used
num_threads = ['none','pragma']

# Execution time for each number of threads
execution_time = [4.0068,3.2319]

# Create a table
table_data = zip(num_threads, execution_time)
headers = ["Number of Threads", "Execution Time (seconds)"]
table = tabulate(table_data, headers=headers, tablefmt="grid")


# Show the table
print(table)

# Show the plot
plt.show()

+---------------------+----------------------------+
| Number of Threads   |   Execution Time (seconds) |
+=====================+============================+
| none                |                     4.0068 |
+---------------------+----------------------------+
| pragma              |                     3.2319 |
+---------------------+----------------------------+
